In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/Users/fl21879/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# define url
url = "https://www.twomlows.com"

In [ ]:
# get HTML content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# get desired HTML tags
selected_tags = ['address', 'cite', 'footer', 'title', 'header']
for tag_name in selected_tags:
    found_tags = soup.find_all(tag_name)
    
    # print tag name and content
    for tag in found_tags:
        print(f"Content of <{tag_name}>:")
        print(tag.text.strip())

Content of <cite>:
Mike and Sue
Content of <cite>:
Karen WilliamsCaldicot
Content of <cite>:
AW
Content of <cite>:
Claire
Content of <cite>:
Graham Dunfield
Content of <cite>:
Mr & Mrs White
Content of <cite>:
Chloe
Content of <cite>:
Carol
Content of <cite>:
Mr & Mrs H
Content of <footer>:
Twomlows Solicitors and Advocates is the trading name of Twomlows Ltdwhich is registered in England and Wales under number 06600174A list of directors is available at the registered office 20 Newport Road, Caldicot NP26 4BQTwomlows Ltd is authorised and regulated by the Solicitors Regulation Authority (SRA) No 653973 












 




CLIENTS NATIONWIDE AND OUR LOCATION ARE CLOSE TO …..Our offices are particularly convenient to Caldicot, Chepstow, Monmouth, Newport, Gloucester, Cheltenham, Cardiff, Raglan, Abergavenny, Crickhowell, Ross-on-Wye & Hereford. 







Regulated By 





 







Contact Us 







 
01291 422753 - Caldicot





 
01291 623323 - Chepstow





 
01600 716200 - Monmouth


